<a href="https://colab.research.google.com/github/ptran1203/fxgan/blob/master/notebooks/classifier_25RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
from google.colab import drive, output
drive.mount('/content/drive')

!rm -rf '/content/gan_project'
!git clone https://github.com/ptran1203/gan_project
!pip install git+https://www.github.com/keras-team/keras-contrib.git

## for Model definition/training
from keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from keras import applications
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.regularizers import l2
import keras.backend as K
from keras_contrib.applications.resnet import ResNet, basic_block
from keras_contrib.applications.densenet import DenseNet
import keras.applications as k_apps
from keras.layers import (
    Input, Dense, Reshape,
    Flatten, Embedding, Dropout,
    BatchNormalization, Activation,
    Lambda,Layer, Add, Concatenate,
    Average,GlobalAveragePooling2D,
    MaxPooling2D, AveragePooling2D,
)
from keras.layers.convolutional import (
    UpSampling2D, Convolution2D,
    Conv2D, Conv2DTranspose
)
from keras.optimizers import Adam, SGD

from tensorflow.examples.tutorials.mnist import input_data
from keras.utils import to_categorical
import tensorflow as tf 
import numpy as np
import pickle

output.clear()
%cd gan_project

In [ ]:
!pip install image-classifiers
from classification_models.keras import Classifiers
from tensorflow.examples.tutorials.mnist import input_data
from keras.preprocessing.image import ImageDataGenerator
import keras.preprocessing.image as iprocess
from mlxtend.plotting import plot_confusion_matrix
import keras
import datetime
from triplet_loss import *
from utils import *
from const import *

import metrics
# from data_augmentation.dataloader import load_gen, load_dataset
from data_augmentation.model import *
import sklearn.metrics as sk_metrics

## Load dataset
# load data
np.random.seed(0)
dataset = 'face'
resolution = 128
large = 0
train_classes = 80


In [ ]:
rst, channel = 128, 3
input_shape = (rst, rst, channel)
feat_dims = 128
from_scratch = False

frozen_block = []
all_scores = {}
experiments = 1
save = True
opts = [
    Option.vgg16,
    # Option.vgg16_st_aug,
    # Option.gan_v2,
    # Option.gan_v1,
    # Option.bagan
]

feature_extractor = 'vgg16'
learning_rate = 1e-5
loss_weights = [1, 0.01]
epochs = 50

for opt in opts:
    result = run(opt, experiments, frozen_block,
                feature_extractor, save, learning_rate,
                loss_weights, epochs, dataset='face',
                 plot_interval=10)

    all_scores[model_map[opt]] = result
# table = metrics.draw_md_table(all_scores)
# print(table)

In [ ]:
embbeding_model = Model(
    inputs = train_model.inputs[0],
    outputs = train_model.get_layer('side_out').get_output_at(-1)
)
classifier = Model(inputs = train_model.inputs[0], outputs = train_model.get_layer('main_out').get_output_at(-1))
# classifier = flatten_model(classifier)
classifier.compile(optimizer='adam', metrics = ['accuracy'],  loss='categorical_crossentropy')
scatter_plot(x_train, y_train, embbeding_model, 'train')
scatter_plot(x_unseen, y_unseen, embbeding_model, 'val')
scatter_plot(x_test, y_test, embbeding_model, 'test')

In [ ]:

def l2_distance(a, b):
    return np.mean(np.square(a - b))

def cosine_sim(a, b):
    return - (np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)))

def cal_sp_vectors(embbeder, supports,k_shot):
    means = []
    x_sp, y_sp = supports
    classes = np.unique(y_sp)
    # perclassid
    per_class_ids = dict()
    ids = np.array(range(len(x_sp)))
    for c in classes:
        per_class_ids[c] = ids[y_sp == c]

    for c in classes:
        imgs = x_sp[per_class_ids[c][:k_shot]]
        # imgs = utils.triple_channels(imgs)
        latent = embbeder.predict(imgs)
        means.append(np.mean(latent, axis=0))
    return np.array(means)
    
def classify_by_metric(embbeder, supports, images, k_shot=1,metric='l2'):
    x_sp, y_sp = supports
    classes = np.unique(y_sp)
    # currently do one-shot classification
    sp_vectors = cal_sp_vectors(embbeder, supports,k_shot)
    vectors = embbeder.predict(triple_channels(images))
    metric_func = l2_distance if metric == 'l2' else cosine_sim
    similiarity = np.array([metric_func(vector, sp_vector) \
                        for vector in vectors \
                        for sp_vector in sp_vectors]).reshape(-1, len(classes))
    pred = np.argmin(np.array(similiarity), axis=1)
    print(pred)
    return pred

def evaluate_by_metric(embbeder, supports, images, labels, k_shot=1,metric='l2'):
    pred = classify_by_metric(embbeder, supports,
                              images,k_shot=k_shot, metric=metric)

    print(labels)
    acc = (pred == labels).mean()
    return acc

# x_train_aug, y_train_aug = load_gen(dataset, 1)
# x_train_aug=triple_channels(x_train_aug)
x_sp_u, x_test_u, y_sp_u, y_test_u = train_test_split(x, y)
evaluate_by_metric(embbeding_model,
                   (x_test_u, y_test_u),
                   x_sp_u, y_sp_u,
                   k_shot=5, metric='cosine')
Counter(y_unseen)

In [ ]:
print("Test")
x, y = pickle_load("/content/drive/My Drive/GAN/dataset/{}/imgs_labels.pkl".format(dataset))
fname = '/content/drive/My Drive/GAN/{}/latent_encoder_{}'.format(dataset, x.shape[1])
embbeding_model = None
with open(fname + '.json', 'r') as f:
    global embbeding_model
    embbeding_model = keras.models.model_from_json(f.read())
    embbeding_model.load_weights(fname + '.h5')
    scatter_plot(x, y, embbeding_model, 'train', 'tsne')